# Find Working Envelope

For this task you will need to demonstrate your ability to give the robot commands and receive observations about the robots state. You will also need to determine what the working envelope for the tip of the pipette is.

To determine the working envelope you will need to move the the pipette to each corner of the cube that forms the working envelope by adjusting the motor velocities for each axis and recording the co-ordinates at each of the 8 points.

Deliverables
- A README.md file describing the environment set up complete with a list of dependancies. Also include the working Envelope of the pipette in this file.
- Well documented code running a simulation in which commands are sent to the robot and observations are returned.
- Optional: A GIF of the robot responding to commands and printing the observations.

In [1]:
import os
import time
os.chdir("./Y2B-2023-OT2_Twin")
from sim_class import Simulation
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
# Initialize simulation with 1 robot and GUI rendering
sim = Simulation(num_agents=1, render=True, rgb_array=True)
sim

In [3]:
# Get robot ID
robotId = sim.robotIds[0]
robotId

1

In [10]:
def get_states(sim, robotId):
    """
    Get the current state information for a robot in the simulation.
    
    Args:
        sim (Simulation): The simulation instance
        robotId (int): ID of the robot to get states for
        
    Returns:
        dict: Dictionary containing:
            - robot_position: [x,y,z] position of robot base (rounded to 3 decimals)
            - pipette_position: [x,y,z] position of pipette tip (rounded to 3 decimals)
            - joint_positions: List of 3 joint angles in radians (rounded to 3 decimals)
            - joint_velocities: List of 3 joint velocities in rad/s (rounded to 3 decimals)
    """
    # Get full state dictionary for the specified robot
    states = sim.get_states()[f"robotId_{robotId}"]
    
    # Extract and round robot base position coordinates
    robot_position = states["robot_position"]
    robot_position = [round(x, 4) for x in robot_position]  # Round to mm precision
    
    # Extract and round pipette tip position coordinates
    pipette_position = states["pipette_position"] 
    pipette_position = [round(x, 4) for x in pipette_position]
    
    # Get positions of the 3 robot joints (shoulder, elbow, wrist)
    # Rounds joint angles to 4 decimal places
    joint_positions = [states["joint_states"][f"joint_{i}"]["position"] for i in range(3)]
    joint_positions = [round(x, 4) for x in joint_positions]
    
    # Get velocities of the 3 robot joints
    # Rounds velocities to 4 decimal places for readability
    joint_velocities = [states["joint_states"][f"joint_{i}"]["velocity"] for i in range(3)]
    joint_velocities = [round(x, 4) for x in joint_velocities]
    
    # Return dictionary with all state information
    return {
        "robot_position": robot_position,      # [x,y,z] coordinates of robot base
        "pipette_position": pipette_position,  # [x,y,z] coordinates of pipette tip
        "joint_positions": joint_positions,    # List of 3 joint angles in radians  
        "joint_velocities": joint_velocities   # List of 3 joint velocities in rad/s
    }

In [12]:
def move_to_target(sim, robotId, target_position, velocity_scale=0.1, tolerance=0.001, timeout=20):

    """
    Move the robot to a target position using velocity control with timeout.
    
    This function combines position control and timeout monitoring into a single optimized 
    implementation. It uses normalized velocities scaled by distance to smoothly move the 
    robot while checking for both position achievement and timeout conditions.

    Args:
        sim (Simulation): The simulation instance
        robotId (int): ID of the robot to move
        target_position (list): [x, y, z] target position coordinates
        velocity_scale (float): Scale factor for velocities (default: 0.1)
        tolerance (float): Distance tolerance for considering position reached (default: 0.001)
        timeout (float): Maximum time in seconds to attempt movement (default: 20)

    Returns:
        bool: True if target reached successfully, False if timeout occurred
    """
    start_time = time.time()
    
    while True:

        # Get current position and calculate distance vector to target
        current_position = get_states(sim, robotId)["pipette_position"]
        distance_vector = [t - c for t, c in zip(target_position, current_position)]
        
        # Check if target reached within tolerance
        if all(abs(d) < tolerance for d in distance_vector):
            return True
            
        # Check timeout condition
        if time.time() - start_time > timeout:
            return False
            
        # Calculate normalized velocities scaled by distance
        max_distance = max(abs(d) for d in distance_vector)
        if max_distance > 0:
            # Scale velocities proportionally to distance while maintaining direction
            velocities = [d * velocity_scale * 10 / max_distance for d in distance_vector]
        else:
            velocities = [0, 0, 0]
            
        # Execute movement step (x_vel, y_vel, z_vel, drop)
        movement = velocities + [0]  # No drop command
        sim.run([movement], num_steps=1)
        
        # Small delay to prevent CPU overload
        time.sleep(0.01)

In [2]:
def find_working_envelope(sim):
    """
    Find the working envelope (reachable space) of the robot by systematically testing movement 
    to points in a cubic grid and recording the actual achieved positions.
    
    The function tests movement to 27 points in a 3x3x3 grid spanning a cubic workspace.
    For each axis, it records the minimum and maximum positions actually reached by the robot,
    which defines the boundaries of the working envelope.
    
    Args:
        sim (Simulation): The simulation instance containing the robot
        
    Returns:
        tuple: Contains the minimum and maximum reachable positions for each axis
               in the order (x_min, x_max, y_min, y_max, z_min, z_max)
    
    Note:
        - Uses normalized workspace coordinates from -1 to 1
        - Tests fewer points than a full workspace analysis but gives a good approximation
        - Actual reachable positions may differ from target positions due to robot constraints
    """
    # Define the workspace boundaries to test
    # Using normalized -1 to 1 range which will be mapped to actual robot workspace
    min_val, max_val = -1, 1
    
    # Get the ID of the first robot in the simulation
    robotId = sim.robotIds[0]

    # Initialize variables to track the envelope boundaries
    # These will store the actual achieved positions at the extremes
    x_max, x_min = None, None  # X-axis boundaries
    y_max, y_min = None, None  # Y-axis boundaries 
    z_max, z_min = None, None  # Z-axis boundaries
    
    # Test movement to points in a 3x3x3 grid
    # Using 3 points per axis gives us the extremes and middle
    for x in np.linspace(min_val, max_val, 3):
        for y in np.linspace(min_val, max_val, 3):
            for z in np.linspace(min_val, max_val, 3):

                # Create target position vector
                target_pos = [x, y, z]
                
                # Attempt to move robot to target position
                move_to_target(sim, robotId, target_pos, velocity_scale=0.1, tolerance=0.001, timeout=20)
                
                # Get actual position achieved by robot
                current_pos = get_states(sim, robotId)["pipette_position"]
                
                # Update envelope boundaries if this point was at an extreme
                if x == min_val: x_min = current_pos[0]  # Minimum x position
                if x == max_val: x_max = current_pos[0]  # Maximum x position
                if y == min_val: y_min = current_pos[1]  # Minimum y position
                if y == max_val: y_max = current_pos[1]  # Maximum y position
                if z == min_val: z_min = current_pos[2]  # Minimum z position
                if z == max_val: z_max = current_pos[2]  # Maximum z position

    # Return tuple containing all envelope boundaries
    return (x_min, x_max, y_min, y_max, z_min, z_max)

# Find and display the working envelope
working_envelope = find_working_envelope(sim)
print("Working envelope (x_min, x_max, y_min, y_max, z_min, z_max):", working_envelope)

Working envelope (x_min, x_max, y_min, y_max, z_min, z_max): (-0.1871, 0.253, -0.1705, 0.2195, 0.1195, 0.2895)
